### IMPORTS

In [1]:
import time
import os
import yaml
import pickle
import json

In [2]:
import ipynb.fs.full.training as training
import ipynb.fs.full.ensemble as ensemble

In [3]:
import tensorflow.python.keras as keras
from tcn import TCN

### STORAGE ROOT

In [4]:
storage_root = 'C://Users/35840/desktop/coding/python/pipeline/storage'

### SAVE PIPELINE

In [5]:
def save_pipeline(params):

    # DECONSTRUCT PARAMS
    config = params['config']
    regression_ensemble = params['regression_ensemble']
    classifier_ensemble = params['classifier_ensemble']
    predictions = params['predictions']
    regression_fitting = params['regression_fitting']
    
    # USE TIMESTAMP AS NAMING CONVENTION
    now = int(time.time())
    
    # PIPELINE DIR NAME
    name = 'PIPELINE-{}'.format(now)
    
    # DIR PATH
    dir_path = '{}/{}'.format(storage_root, name)
    
    # CREATE NEW DIR
    os.mkdir(dir_path)
    
    # CREATE A PREDICTIONS DIR
    os.mkdir('{}/predictions'.format(dir_path))
    
    # SERIALIZE & SAVE THE YAML CONFIG
    save_json(config, '{}/config.json'.format(dir_path))
    
    # SERIALIZE & SAVE PREDICTIONS & REGERSSION FITTING
    save_json(predictions, '{}/predictions.json'.format(dir_path))
    save_json(regression_fitting, '{}/regression_fitting.json'.format(dir_path))
    
    # SAVE REGRESSION ENSEMBLE
    save_ensemble(regression_ensemble, 'regression', dir_path)
    
    # SAVE CLASSIFIER ENSEMBLE
    save_ensemble(classifier_ensemble, 'classifier', dir_path)
    
    return name

### SAVE A PIPELINE PREDICTION

In [46]:
def save_prediction(pipeline_name, predictions):
    
    # DIR PATH
    dir_path = '{}/{}'.format(storage_root, pipeline_name)
    
    # CURRENT TIMESTAMP
    now = int(time.time())
    
    # REFORMAT THE PREDICTIONS
    formatted = {
        'graph': 'line',
        'data': json.loads(predictions.to_json())
    }
    
    # PARSE & SAVE AS JSON
    save_json(formatted, '{}/predictions/{}.json'.format(dir_path, now))
    
    return now

### SAVE ENSEMBLE

In [7]:
def save_ensemble(ensemble, name, root):
    
    # SUBPATH
    path = '{}/{}_ensemble'.format(root, name)
    
    # CREATE SUBDIR
    os.mkdir(path)
    
    for model in ensemble.models:
        
        # MANY MODELS
        if type(model) == list:
            sub_path = path + '/' + model[0].name + '/'
            os.mkdir(sub_path)
            
            for index, sub in enumerate(model):
                final_path = sub_path + str(index)
                os.mkdir(final_path)
                sub.save(final_path)
                
        else:
            sub_path = path + '/' + model.name + '/'
            os.mkdir(sub_path)
            model.save(sub_path)

### SAVE/LOAD YAML DATA

In [8]:
def load_yaml(path):
    with open(path, mode='r') as file:
        return yaml.load(file, Loader=yaml.FullLoader)

### SAVE DATAFRAME AS CSV

In [9]:
def save_csv(dataframe, path):
    dataframe.to_csv(path)

### SAVE PICKLE

In [10]:
def save_pickle(data, path):
    pickle.dump(data, open(path, 'wb'))

In [11]:
def load_pickle(path):
    return pickle.load(open(path, 'rb'))

### LOAD & SAVE AS JSON

In [12]:
def load_json(path):
    with open(path) as json_file:
        return json.load(json_file)

In [13]:
def save_json(data, path):
    with open(path, 'w') as outfile:
        json.dump(data, outfile)

### LOAD PIPELINE

In [14]:
def load_pipeline(pipeline_name):
    
    # DIR PATHS
    root_path = '{}/{}'.format(storage_root, pipeline_name)
    reg_path = '{}/regression_ensemble'.format(root_path)
    cls_path = '{}/classifier_ensemble'.format(root_path)
    
    # SERIALIZE THE CONFIG FILE
    config = load_json('{}/config.json'.format(root_path))
    
    # MODEL NAMES
    reg_models = os.listdir(reg_path)
    cls_models = os.listdir(cls_path)
    
    # ENSEMBLES
    reg_ensemble = ensemble.create_ensemble()
    cls_ensemble = ensemble.create_ensemble()
        
    # SERIALIZE REGRESSION MODELS
    for model_name in reg_models:
        temp_path = '{}/{}/'.format(reg_path, model_name)
        collection = []
        
        # PRINT MESSAGE
        print('SERIALIZING REGRESSOR {}'.format(model_name.upper()))

        # SERIALIZE & APPEND SUBMODELS TO COLLECTION
        for sub_model in os.listdir(temp_path):
            fin_path  = '{}/{}/'.format(temp_path, sub_model)
            model = build_model(fin_path, model_name)
            collection.append(model)
            
            # PRINT MESSAGE
            print('\tFOLD {}'.format(sub_model.upper()))
            
        # ADD THE COLLECTION TO THE ENSEMBLE
        reg_ensemble.add_model(collection)
        print()
    
    # SERIALIZE CLASSIFIER MODELS
    for model_name in cls_models:
        temp_path = '{}/{}/'.format(cls_path, model_name)
        
        # BUILD THE MODEL & ADD IT TO THE ENSEMBLE
        model = build_model(temp_path, model_name)
        cls_ensemble.add_model(model)
        
        # PRINT MESSAGE
        print('SERIALIZED CLASSIFIER {}'.format(model_name.upper()))
            
    return reg_ensemble, cls_ensemble, config

### BUILD A MODEL

In [15]:
def build_model(path, model_name):
    files = set(os.listdir(path))

    # LOAD THE SCALER & SETTINGS
    scaler = load_pickle(path + 'scaler.pickle')
    settings = load_pickle(path + 'settings.pickle')

    # LOAD SKLEARN MODEL
    if 'model.pickle' in files:
        model = load_pickle(path + 'model.pickle')
        model = training.basic_model(model, model_name, settings, scaler)

    # LOAD KERAS MODEL
    else:
        model = keras.models.load_model(path + 'model.keras', custom_objects={ 'TCN': TCN })
        model = training.generator_model(model_name, settings, scaler, model)

    return model

### LOAD PIPELINE DETAILS

In [42]:
def load_details(pipeline_name):
    
    # PATH
    root_path = '{}/{}'.format(storage_root, pipeline_name)
    
    # CONTAINER
    results = {}
    
    # LOOP THROUGH PREDICTION RESULTS
    for file in os.listdir('{}/predictions'.format(root_path)):
        
        # LOAD CONTENT & EXTRACT NAME
        data = load_json('{}/predictions/{}'.format(root_path, file))
        file_name = file.split('.')[0]
        
        # PUSH TO RESULTS
        results[file_name] = data
    
    return {
        'predictions': load_json('{}/predictions.json'.format(root_path)),
        'regression_fitting': load_json('{}/regression_fitting.json'.format(root_path)),
        'config': load_json('{}/config.json'.format(root_path)),
        'results': results
    }